In [8]:
import pandas as pd
import json

In [4]:
# Dados da Netflix
netflix_raw = pd.read_csv('https://raw.githubusercontent.com/Cizika/netflix-analysis/main/netflix_titles.csv', index_col=0)
netflix_raw.head()

,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
show_id,,,,,,,,,,,
s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [11]:
# Carregando Dados de países/continentes
with open('countries.json') as json_file:
    countries = json.load(json_file)

In [12]:
# Lista de países europeus
europa_list = countries["Europe"]["countries"]
europa_list

['Albania',
 'Andorra',
 'Åland',
 'Austria',
 'Belarus',
 'Belgium',
 'Bosnia and Herzegovina',
 'Bulgaria',
 'Croatia',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Estonia',
 'Faroe Islands',
 'Finland',
 'France',
 'Germany',
 'Gibraltar',
 'Greece',
 'Greenland',
 'Guernsey',
 'Hungary',
 'Iceland',
 'Ireland',
 'Isle of Man',
 'Israel',
 'Italy',
 'Jersey',
 'Latvia',
 'Liechtenstein',
 'Lithuania',
 'Luxembourg',
 'Macedonia',
 'Malta',
 'Moldova',
 'Monaco',
 'Montenegro',
 'Netherlands',
 'Norway',
 'Poland',
 'Romania',
 'Russia',
 'Serbia',
 'Spain',
 'Sweden',
 'Switzerland',
 'Slovakia',
 'Slovenia',
 'Svalbard and Jan Mayen',
 'San Marino',
 'Ukraine',
 'United Kingdom']

In [13]:
# Lista de países latinos
latin_list = countries["Latin America"]["countries"]
latin_list

['Costa Rica',
 'Cuba',
 'Haiti',
 'Panama',
 'Saint Bartélemy',
 'El Salvador',
 'Saint Martin',
 'Argentina',
 'Belize',
 'Bolivia',
 'Brazil',
 'Chile',
 'Colombia',
 'Ecuador',
 'Dominican Republic',
 'Guatemala',
 'Honduras',
 'Mexico',
 'Nicaragua',
 'Paraguay',
 'Peru',
 'Puerto Rico',
 'Uruguay',
 'Venezuela']

In [13]:
# Limpeza Dataframe
df = df_raw[["type", "country","date_added"]].dropna()
df

,type,country,date_added
show_id,,,
s1,Movie,United States,"September 25, 2021"
s2,TV Show,South Africa,"September 24, 2021"
s5,TV Show,India,"September 24, 2021"
s8,Movie,"United States, Ghana, Burkina Faso, United Kin...","September 24, 2021"
s9,TV Show,United Kingdom,"September 24, 2021"
...,...,...,...
s8802,Movie,"United Arab Emirates, Jordan","March 9, 2016"
s8803,Movie,United States,"November 20, 2019"
s8805,Movie,United States,"November 1, 2019"


In [14]:
# Coletar amostra dos dados com data de adição em 2021
df["date_added"] = df["date_added"].apply(lambda x: int(x.split(", ")[1]))

amostra = df[df["date_added"] == 2021]
amostra

,type,country,date_added
show_id,,,
s1,Movie,United States,2021
s2,TV Show,South Africa,2021
s5,TV Show,India,2021
s8,Movie,"United States, Ghana, Burkina Faso, United Kin...",2021
s9,TV Show,United Kingdom,2021
...,...,...,...
s8273,Movie,"United States, Hong Kong",2021
s8321,Movie,"United States, Sweden, Norway",2021
s8435,Movie,United States,2021


In [15]:
# Dicotomizar por País de Origem Latino vs. Não-Latino
latin_countries = ["Brazil", "Argentina", "Chile", "Colombia", "Venezuela", "Guatemala", "Mexico", "Honduras", 
"Costa Rica", "Peru", "Dominican Republic", "Jamaica", "Paraguay", "Uruguay", "Trinidad and Tobago", "Cuba", "Bolivia", 
"Ecuador", "Suriname", "Belize"]

# Adicionando Coluna is_latin se o país de origem é da América Latina
df["is_latin"] = df["country"].isin(latin_countries)
df

,type,country,date_added,is_latin
show_id,,,,
s1,Movie,United States,2021,False
s2,TV Show,South Africa,2021,False
s5,TV Show,India,2021,False
s8,Movie,"United States, Ghana, Burkina Faso, United Kin...",2021,False
s9,TV Show,United Kingdom,2021,False
...,...,...,...,...
s8802,Movie,"United Arab Emirates, Jordan",2016,False
s8803,Movie,United States,2019,False
s8805,Movie,United States,2019,False


In [16]:
# Contagem de quantos filmes latinos foram adicionados em 2021
df[df["is_latin"] == True].count()

type          301
country       301
date_added    301
is_latin      301
dtype: int64